Goal: Correlation between stock prices and the sentiment of tweets of said stocks

Importing needed libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Data Preprocessing and Feature Engineering
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

Reading the data with Pandas

In [5]:
training_tweets = pd.read_csv('training_tweets.csv')
testing_tweets = pd.read_csv('testing_tweets_tsla.csv')